In [ ]:
import sqlite3

# Connect to SQLite database
database_path="Datasets/database/longlist.db"
conn = sqlite3.connect(database_path)

# Create a cursor object
cursor = conn.cursor()

```mermaid
erDiagram
  collections {
    INTEGER id PK
    TEXT title
    TEXT accession_number
    NUMERIC acquired
  }
  artists {
    INTEGER id PK
    TEXT name
  }
  created {
    INTEGER artist_id PK
    INTEGER collection_id PK
  }
  created ||--o{ collections : "collection_id → id"
  created ||--|| artists : "artist_id → id"
```

In [ ]:
# Start Mermaid diagram
mermaid_output = ["erDiagram"]

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

for table in tables:
    # Get column details
    cursor.execute(f"PRAGMA table_info({table});")
    columns = cursor.fetchall()  # (cid, name, type, notnull, dflt_value, pk)
    
    # Define table in Mermaid format
    mermaid_output.append(f"  {table} {{")
    column_dict = {}  # Store column details for FK check
    for col in columns:
        col_name = col[1]
        col_type = col[2].upper()
        pk_flag = " PK" if col[5] else ""
        mermaid_output.append(f"    {col_type} {col_name}{pk_flag}")
        column_dict[col_name] = col[5]  # Track if column is a PK
    
    mermaid_output.append("  }")
    
    # Get foreign key relationships
    cursor.execute(f"PRAGMA foreign_key_list({table});")
    foreign_keys = cursor.fetchall()
    
    for fk in foreign_keys:
        from_column = fk[3]  # FK column in current table
        to_table = fk[2]      # Referenced table
        to_column = fk[4]     # Referenced column
        
        # Check if the FK column is also a primary key (1:1 relationship)
        if column_dict.get(from_column, 0) == 1:
            mermaid_output.append(f"  {table} ||--|| {to_table} : \"{from_column} → {to_column}\"")  # 1:1
        else:
            mermaid_output.append(f"  {table} ||--o{{ {to_table} : \"{from_column} → {to_column}\"")  # 1:M

# Print Mermaid syntax
mermaid_code = "\n".join(mermaid_output)
print(mermaid_code)

# Close connection
conn.close()
